# Recuperação de dados no Twitter

Este notebook tem como objetivo demostrar a recuperação de dados no tweeter, via API e persistir as informações em documentos para posterior uso.

## Imports

Vamos utilizar nesse exemplo as bibliotecas:

* os: Para acesso aos recursos do sistema operacional
* tweepy: Para acesso ao twitter (login, coleta)
* pandas: Para manipulação dos dados no formato tabular

In [1]:
import os
import tweepy
import pandas as pd

## Download do idioma

Vamos fazer o download do idioma para manipulação dos textos

## Carregamento das chaves de acesso

As chaves de acesso podem ser geradas através do portal do desenvolvedor do Twitter.

* Site do Twitter: https://twitter.com/ (Links para um site externo.)

* Área de desenvolvedor https://developer.twitter.com/en/apply/user (Links para um site externo.)

* Área de aplicativos https://developer.twitter.com/en/apps (Links para um site externo.)


In [2]:
consumer_key =  os.environ['tweety_consumer_key']
consumer_secret = os.environ['tweety_consumer_secret']
access_token = os.environ['tweety_access_token']
acess_token_secret = os.environ['tweety_acess_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, acess_token_secret)
api = tweepy.API(auth)

## Dataframe

Vamos utilizar um dataframe para persistir nossos registros obtidos a partir do stream de twetts. O Dataframe é similar a uma planilha.

In [3]:
df = pd.DataFrame(columns=['name', 'description', 'created_at', 'text', 'location', 'retweeted', 'owner_name', 'owner_location', 'owner_created_at', 'retweeted_count'])
df.head()

,name,description,created_at,text,location,retweeted,owner_name,owner_location,owner_created_at,retweeted_count


## Streamlister - Execute esse bloco se você não tiver o dataset salvo ainda

Vamos criar um stream listener que ira receber os twitts de acordo com a pesquisa que fizermos. Para isso vamos extender uma classe do pacote tweepy e fazer nossa implementação.

In [4]:
class StreamListner(tweepy.StreamListener):
    def __init__(self, df):
        super().__init__()
        self.__df__ = df

    def on_status(self, status):
        json = dict(status._json)
        user = json['user']
        
        retweety = json.get('retweeted_status')
        retweety_user = retweety['user'] if retweety != None and retweety.get('user') != None else None

        obj = { 'name': user['name'], 'description': user['description'], 'created_at': json['created_at'], 'text': json['text'], 'location': user.get('location'), 'retweeted': retweety != None }

        if retweety != None:
            obj['owner_name'] = retweety_user['name']
            obj['owner_location'] = retweety_user.get('location')
            obj['owner_created_at'] = retweety['created_at']
            obj['retweeted_count'] = retweety.get('retweet_count', 0)
        
        self.__df__ = self.__df__.append(obj, ignore_index=True)
        

    def on_error(self, status_code):
        if status_code == 420:
            return False

    @property
    def df(self):
        return self.__df__

### Definir o(s) termo(s) de pesquisa.

Podemos definir os n termos que serão utilizados para pesquisa

In [5]:
track = ['covid19', 'covid-19', 'sars-cov-2', 'coronavirus', 'corona-virus']

Criamos o objeto responsável pela busca dos tweets

In [9]:
stream_listner = StreamListner(df)
stream = tweepy.Stream(auth=api.auth, listener=stream_listner)
stream.filter(track=track, languages=['pt'], is_async=True)

In [10]:
stream.disconnect()

## Visualização dos registros - Obtidos pelo stream

Vamos visualizar os registros obtidos pelo stream atraves do dataframe

In [11]:
df = stream_listner.df

In [12]:
df

,name,description,created_at,text,location,retweeted,owner_name,owner_location,owner_created_at,retweeted_count
0,SôniaHenricar,Direita🇧🇷🇧🇷 conservadora. 100% Bolsonaro. Cris...,Mon Feb 01 13:55:27 +0000 2021,RT @AiltonBenedito: Por que o Ministério da Sa...,Brasil,True,🇧🇷Ailton Benedito,Brasil,Mon Feb 01 13:30:25 +0000 2021,90
1,Rogério Araujo - VACINA PARA TODOS,Radical de CENTRO!,Mon Feb 01 13:55:29 +0000 2021,RT @jdoriajr: Iniciativa fundamental para que ...,"Salvador, Brasil",True,João Doria,"São Paulo, SP, Brasil",Mon Feb 01 12:24:14 +0000 2021,15
2,Sherlock narigudo,Quarentena me fez criar Twiter\nInstagram: @_g...,Mon Feb 01 13:55:30 +0000 2021,hj é um bom dia,None,False,NaN,NaN,NaN,NaN
3,MUTIRÃO FORA KERLINE,ela/dela.,Mon Feb 01 13:55:34 +0000 2021,se num fosse coronavirus a gente já tava tendo...,isa,False,NaN,NaN,NaN,NaN
4,Expresso,Toda a informação do site do jornal de referên...,Mon Feb 01 13:55:35 +0000 2021,Covid-19. Nova variante identificada no México...,"Lisboa, Portugal",False,NaN,NaN,NaN,NaN
5,Dr. Lette,"Olá, eu sou o Dr. Lette. Sou a cara do Brasil:...",Mon Feb 01 13:55:35 +0000 2021,"da série ""Hipocrisia Brasileira""\n#ChinaVirus ...",brasilia,False,NaN,NaN,NaN,NaN
6,.,o pior perfil do twitter inteiro http://instag...,Mon Feb 01 13:55:38 +0000 2021,RT @transacerta: Sem alarde família to imuniza...,@afterspring_,True,1renas.{IMP} ☯️,Lado A do ABC,Mon Feb 01 13:16:52 +0000 2021,10
7,🔰 LAVAJATO AVANTE- FORA BOLSOPETISTA,Vamos unir em defesa da lava-jato -Não tenho p...,Mon Feb 01 13:55:38 +0000 2021,RT @VillaMarcovilla: Bolsonaro leva o Brasil p...,"São Paulo, Brasil",True,Marco Antonio Villa,São Paulo - Brasil,Mon Feb 01 03:48:47 +0000 2021,133
8,Robô politicamente incorreto,APENAS UM ROBÔ PROGRAMADO PARA APONTAR TRAÍRAS...,Mon Feb 01 13:55:42 +0000 2021,RT @tisemherois: @STF_oficial De que obscurant...,None,True,Sergio Silva,"-23.539341,-46.478732",Mon Feb 01 13:52:07 +0000 2021,1
9,Pietro Bernhardt Carvalho,None,Mon Feb 01 13:55:43 +0000 2021,"RT @alexaguiarpoa: EXTRAORDINÁRIO - Somente 0,...","Rio Grande, Brasil",True,Alexandre Aguiar,"Porto Alegre, Brasil",Mon Feb 01 01:39:11 +0000 2021,63
